## Timetabling Program for St. George's School
#### Copyright  © 2020 Hoshino Math Services

In [1]:
"""This script consists of an Hill Climber heuristic, which
generates a close-to-optimal timetable for grade 11/12 Students at St.George's School 
for the academic year 2020-2021, assuming a post-COVID world with cohort partitioning."""

# Import Python Modules

# import sys
# !{sys.executable} -m pip install ortools 


# import libraries
import time
import numpy as np
import pandas as pd
import random
from random import shuffle
from ortools.linear_solver import pywraplp

### The Data

In [2]:
# Import Student and Course Data 

InputFile = pd.ExcelFile("Input Data.xlsx") 
StudentMatrix = pd.read_excel(InputFile, 'Student Selections') 
StudentInfo = StudentMatrix.values.tolist()
CourseMatrix = pd.read_excel(InputFile, 'Sections') 
CourseInfo = CourseMatrix.values.tolist()


In [3]:
# Generate the list of courses, list of students, and class lists
# NOTE: Career Life Connections = 99, Spare 12 = 231, Spare 11 = 232
    
CourseIDList = [99, 231, 232]
for i in range(len(CourseInfo)):
    CourseID = CourseInfo[i][1]
    if not CourseID in CourseIDList:
        CourseIDList.append(CourseID)
CourseIDList.sort()

StudentIDList = []
for i in range(len(StudentInfo)):
    StudentID = StudentInfo[i][0]
    if not StudentID in StudentIDList:
        StudentIDList.append(StudentID)
StudentIDList.sort()

n = len(StudentIDList)
m = len(CourseIDList)

StudentChoices = [ [] for x in range(n)]
for i in range(len(StudentInfo)):
    StudentID = StudentInfo[i][0]
    StudentIndex = StudentIDList.index(StudentID)
    CourseName = StudentInfo[i][4]
    if CourseName in CourseIDList:
        CourseIndex = CourseIDList.index(CourseName)
        if not CourseIndex in StudentChoices[StudentIndex]:
            StudentChoices[StudentIndex].append(CourseIndex)

            
# Generate the list of grade 11, grade 12, and boarding students          
Grade11Students = []
Grade12Students = []
BoardingStudents = []

for i in range(len(StudentInfo)):
    StudentID = StudentInfo[i][0]
    StudentIndex = StudentIDList.index(StudentID)    
    if StudentInfo[i][1] == 'Grade 11':
        Grade11Students.append(StudentIndex) 
    if StudentInfo[i][1] == 'Grade 12':
        Grade12Students.append(StudentIndex) 
    if StudentInfo[i][2] == 'Boarding':
        BoardingStudents.append(StudentIndex)

Grade11Students = list(set(Grade11Students))
Grade11Students.sort()  
Grade12Students = list(set(Grade12Students))
Grade12Students.sort()  
BoardingStudents = list(set(BoardingStudents))
BoardingStudents.sort()   

CourseNameList = ['' for i in range(m)]
for i in range(len(CourseInfo)):
    CourseName = CourseInfo[i][0]
    CourseID = CourseInfo[i][1]
    CourseIndex = CourseIDList.index(CourseID)
    CourseNameList[CourseIndex] = CourseName
    
ClassList = [ [] for x in range(m)]
Enrollment = [ 0 for x in range(m)]
for i in range(n):
    for j in range(len(StudentChoices[i])):
        CourseIndex = StudentChoices[i][j]
        Enrollment[CourseIndex] += 1
        if not i in ClassList[CourseIndex]:
            ClassList[CourseIndex].append(i)  

            
# Generate lists of courses divided by priority (Must, High, Medium, Low)            
MustPriority = []
HighPriority = []
MedPriority = []
TripletList = [ [] for t in range(len(CourseInfo))]
TripletCap = [ 0 for t in range(len(CourseInfo))]
CombinedLG = []

indexcount = 0

for i in range(len(CourseInfo)):

    CourseID = CourseInfo[i][1]
    CourseIndex = CourseIDList.index(CourseID)    
    Section = CourseInfo[i][4]
    Block = CourseInfo[i][5]
    Capacity = CourseInfo[i][6]
    TripletList[indexcount] = [CourseIndex, Section, Block]
    
    TripletCap[indexcount] = Capacity
    if CourseInfo[i][11] == 'n' or CourseInfo[i][11] == 'N':
        TripletCap[indexcount] = 0    
        
    if CourseInfo[i][10] == 'y' or CourseInfo[i][10] == 'Y':
        CombinedLG.append(indexcount)
 
    indexcount += 1
        
    if CourseInfo[i][7] == 'Y': 
        if not CourseIndex in HighPriority:
            if not CourseInfo[i][13] == 'Y' and not CourseInfo[i][13] == 'y':
                HighPriority.append(CourseIndex)
    if CourseInfo[i][8] == 'Y':
        if not CourseIndex in MedPriority:
            MedPriority.append(CourseIndex)        
    if CourseInfo[i][13] == 'Y' or CourseInfo[i][13] == 'y':
        if not CourseIndex in MustPriority:
            MustPriority.append(CourseIndex) 
        

In [4]:
# Find all the Triplets that belong to the same course, and to the same block

Blocks = ['A','B','C','D','E','F','G','H']

TripletIDs = [ [] for r in range(m)]
TripletBlocks = [ [] for r in range(8)]

for t in range(len(TripletList)):
    ID = TripletList[t][0]
    Block = TripletList[t][2]
    TripletIDs[ID].append(t)
    for r in range(8):
        if Blocks[r]==Block:
            TripletBlocks[r].append(t)

In [5]:
# Manually list out all the pairs of rows (t) that must be assigned to the same Learning Group

SameLGList = [[38,41], [38,43], [41,43], [39,42], [39,44], [42,44], [40,45], [46,48], [47,49],
              [166,168], [167,169], [170,172], [171,173], [154,155], [194,195]]

print("These course sections are to be combined into the same unique Learning Group")
print("")
for t in SameLGList:
    if not t[0] in CombinedLG:
        print(TripletList[t[0]][1], t, CourseNameList[TripletList[t[0]][0]])
        print(TripletList[t[1]][1], t, CourseNameList[TripletList[t[1]][0]])
        print("")
    

These course sections are to be combined into the same unique Learning Group

2H [154, 155] Spanish 11
1H [154, 155] Spanish 12

1E [194, 195] Yearbook Media Design 11
1E [194, 195] Yearbook Media Design 12



In [6]:
print("The following courses were requested by ZERO students:")
print("")
for j in range(m):
    count=0
    for i in range(n):
        if j in StudentChoices[i]:
            count+=1
    if count==0:
        print(CourseIDList[j], CourseNameList[j])

The following courses were requested by ZERO students:

164 Directed Studies 12
275 Latin 12
317 Graphics 12
347 Journalism 12
372 Literary Studies 12
438 Composition 12
439 Writers Workshop 11
466 Active Living 11
468 Active Living 12
477 French 12 (Honours) (S2)
736 AP 3-D Design Portfolio 12
738 AP 2-D Design Portfolio 12
800 Economics 12 AP (Macro) (S2)


In [7]:
print("The following single-section courses have more students than the capacity:")
print("")
for j in range(m):
    if len(TripletIDs[j]) == 1:
        t = TripletIDs[j][0]
        Capacity = TripletCap[t]
        if Capacity < Enrollment[j]:
            print(CourseNameList[j], "is requested by", Enrollment[j],
                  "but has a capacity of", Capacity)   
print("")
print("")
print("The following multi-section courses have more students than the capacity:")
print("")
for j in range(m):
    if len(TripletIDs[j]) > 1:
        Sections = len(TripletIDs[j])
        Capacity = sum(TripletCap[t] for t in TripletIDs[j])
        if Capacity < Enrollment[j]:
            print(Sections, "Sections:", CourseNameList[j], "is requested by", Enrollment[j], 
                  "but has a capacity of", Capacity)
    



The following single-section courses have more students than the capacity:

Biology 12 AP is requested by 23 but has a capacity of 18
Journalism 11 is requested by 1 but has a capacity of 0
Comparative Cultures 12 is requested by 17 but has a capacity of 15
Robotics 11 is requested by 1 but has a capacity of 0
Robotics 12 is requested by 1 but has a capacity of 0
Studio Arts 3D 12 is requested by 22 but has a capacity of 16
Asian Studies 12 is requested by 5 but has a capacity of 0
Advanced Topics in Mathematics 12 is requested by 5 but has a capacity of 0
World History 12 AP is requested by 23 but has a capacity of 20
Yearbook Media Design 11 is requested by 3 but has a capacity of 0
Yearbook Media Design 12 is requested by 4 but has a capacity of 0


The following multi-section courses have more students than the capacity:

2 Sections: Chemistry 12 is requested by 45 but has a capacity of 44
3 Sections: Chemistry 12 AP is requested by 45 but has a capacity of 38
2 Sections: Gov't / P

In [8]:
# Generate the set of courses requested by Grade 11/12 students
# We do not include Spare or Career-Life or any classes with zero enrollment
# or any courses cancelled by Sarah, the Associate Principle of Academics at SGS

ActualCourses = []
for j in range(m):
    if Enrollment[j]>0 and len(TripletIDs[j]) > 0:
        ActualCourses.append(j)
        
LowPriority = list(set(ActualCourses)-set(HighPriority)-set(MedPriority)-set(MustPriority))        
AllSingles = []
AllMultiples = []

for j in ActualCourses:
    if len(TripletIDs[j]) == 1: AllSingles.append(j)
    if len(TripletIDs[j]) > 1: AllMultiples.append(j)

print("There are", n, "Grade 11/12 students requesting a total of",
     len(ActualCourses), "courses")
print("Of these", len(ActualCourses), "courses,", len(AllSingles), "of them are singletons and",
     len(AllMultiples), "have multiple sections")
print("We are not scheduling Career Life Connections")

There are 328 Grade 11/12 students requesting a total of 89 courses
Of these 89 courses, 41 of them are singletons and 48 have multiple sections
We are not scheduling Career Life Connections


In [9]:
print("All Grade 11 students must have one of these English courses")
for j in [54, 89, 55, 90]:
    if j in MustPriority:incipit = 'MUST -'
    else: incipit = ''
    print(incipit, CourseIDList[j], CourseNameList[j])

print()
print("All Grade 12 students must have one of these English courses")
for j in [46, 49, 56]:
    if j in MustPriority: incipit = 'MUST -'
    else: incipit = ''
    print(incipit, CourseIDList[j], CourseNameList[j])
    
print()
print("All Grade 11 students must get into their language course")
for j in [29, 30, 35, 92, 91]:
    if j in MustPriority: incipit = 'MUST -'
    if j in HighPriority: incipit = 'HIGH -'
    if j in MedPriority: incipit = 'MEDIUM -'
    if j in LowPriority: incipit = 'LOW -'
    print(incipit, CourseIDList[j], CourseNameList[j])

All Grade 11 students must have one of these English courses
MUST - 368 Composition 11
MUST - 492 Composition 11 Honours
MUST - 370 Literary Studies 11
MUST - 493 Literary Studies 11 (Honours)

All Grade 12 students must have one of these English courses
MUST - 337 English 12 AP (Language)
MUST - 353 English 12 AP (Literature)
MUST - 371 English Studies 12

All Grade 11 students must get into their language course
LOW - 261 French 12 (Honours)
LOW - 262 French 12 AP
LOW - 281 Mandarin 12
LOW - 648 Mandarin 12 AP
MUST - 645 Pre-Calculus 12 - Gr 11 (Competition)


In [10]:
# In the timetable we sent to SGS, we got 807 out of 808 must courses satisfied.
# The one exception was student i=243 who requested j=90.  We give that student j=55
# instead, replacing Literary Studies 11 (Honours) with Literary Studies 11

StudentChoices[243].remove(90)
StudentChoices[243].append(55)

### The Preference Matrix

In [11]:
# Set preference coefficients
# Assume weight of 5-3-1 for high/medium/low courses

P = np.zeros((n,m), dtype=int)

for i in range(n):
    for j in StudentChoices[i]:
        if not j in [15,16]:
            P[i,j] = 1
            if j in MustPriority: P[i,j] = 10
            if j in HighPriority: P[i,j] = 5   
            if j in MedPriority: P[i,j] = 3
        
P.shape

(328, 103)

In [12]:
# Count the theoretical maximum happiness score for priority courses
# Note that the Low count is off (789 vs. 1165) due to the Spare 11 and Spare 12 course, and the
# Career Life course that takes place outside of school hours.

MustOccurrences = sum(np.count_nonzero(P == 10, axis=1))
MustMaxScore = 10*sum(np.count_nonzero(P == 10, axis=1))
HighOccurrences = sum(np.count_nonzero(P == 5, axis=1))
HighMaxScore = 5*sum(np.count_nonzero(P == 5, axis=1))
MedOccurrences = sum(np.count_nonzero(P == 3, axis=1))
MedMaxScore = 3*sum(np.count_nonzero(P == 3, axis=1))
LowOccurrences = sum(np.count_nonzero(P == 1, axis=1))
LowMaxScore = 1*sum(np.count_nonzero(P == 1, axis=1))
AllAssignments = MustOccurrences+HighOccurrences+MedOccurrences+LowOccurrences
TheoreticalMax = MustMaxScore+HighMaxScore+MedMaxScore+LowMaxScore
MaximumScore = sum(sum(P[i][j] for j in range(m)) for i in range(n))


print('Total course requests:', AllAssignments, ' w/ theoretical maximum happiness score:',  TheoreticalMax)
print('Total Must Priority course requests:', MustOccurrences, ' w/ happiness score:',  MustMaxScore)
print('Total High Priority course requests:', HighOccurrences, ' w/ happiness score:',  HighMaxScore)
print('Total Medium Priority course requests:', MedOccurrences, ' w/ happiness score:',  MedMaxScore)
print('Total Low Priority course requests:', LowOccurrences, ' w/ happiness score:',  LowMaxScore)
print(sum(np.count_nonzero(P != 0, axis = 1))==AllAssignments, TheoreticalMax==MaximumScore)

Total course requests: 2466  w/ theoretical maximum happiness score: 12225
Total Must Priority course requests: 807  w/ happiness score: 8070
Total High Priority course requests: 541  w/ happiness score: 2705
Total Medium Priority course requests: 166  w/ happiness score: 498
Total Low Priority course requests: 952  w/ happiness score: 952
True True


In [13]:
# Create a list of Priority Course Indeces
Triplets = range(len(TripletList))
MustPriorityTriplets = []
HighPriorityTriplets = []
MedPriorityTriplets = []
LowPriorityTriplets = []

for t in Triplets:
    if TripletList[t][0] in MustPriority:
        MustPriorityTriplets.append(t)
    elif TripletList[t][0] in HighPriority:
        HighPriorityTriplets.append(t)
    elif TripletList[t][0] in MedPriority:
        MedPriorityTriplets.append(t)
    else:
        LowPriorityTriplets.append(t)
        
grandTotal = len(MustPriorityTriplets)+len(HighPriorityTriplets)+len(MedPriorityTriplets)+len(LowPriorityTriplets)
print(len(MustPriorityTriplets), '+', len(HighPriorityTriplets), '+', 
      len(MedPriorityTriplets), '+', len(LowPriorityTriplets), '=', len(Triplets), '=', grandTotal)
len(Triplets)==grandTotal

print('There are', len(MustPriorityTriplets), 'Must Priority Triplets')
print('There are', len(HighPriorityTriplets), 'High Priority Triplets')
print('There are', len(MedPriorityTriplets), 'Medium Priority Triplets')
print('There are', len(LowPriorityTriplets), 'Low Priority Triplets')
print('There are', len(Triplets), 'Triplets in total')

47 + 32 + 19 + 98 = 196 = 196
There are 47 Must Priority Triplets
There are 32 High Priority Triplets
There are 19 Medium Priority Triplets
There are 98 Low Priority Triplets
There are 196 Triplets in total


In [14]:
# Partition the Must Priority Course Indeces into blocks
MustPriorityTripletBlocks = [ [] for r in range(8)]
MustHighTripletBlocks = [ [] for r in range(8)]
MustHighMedTripletBlocks = [ [] for r in range(8)]

for r in range(8):
    for b in TripletBlocks[r]:
        if b in MustPriorityTriplets:
            MustPriorityTripletBlocks[r].append(b)
        if b in MustPriorityTriplets or b in HighPriorityTriplets:
            MustHighTripletBlocks[r].append(b)
        if b in MustPriorityTriplets or b in HighPriorityTriplets or b in MedPriorityTriplets:
            MustHighMedTripletBlocks[r].append(b)
            
MustHighMedTripletBlocks

[[27, 31, 32, 33, 64, 89, 91, 113, 133, 145, 153],
 [13, 14, 17, 19, 22, 53, 66, 131, 178, 179],
 [5, 6, 15, 20, 23, 25, 46, 48, 185, 187, 190, 191],
 [26, 28, 65, 67, 69, 70, 79, 80, 81, 82, 135, 137, 140, 141, 144],
 [7, 16, 18, 71, 112, 128, 129, 132, 146],
 [21, 24, 47, 49, 55, 56, 130, 134, 147, 186, 188],
 [29, 68, 72, 73, 83, 84, 85, 86, 138, 139, 142, 143],
 [8, 9, 10, 12, 34, 35, 36, 37, 54, 74, 90, 114, 115, 116, 118, 136, 154, 180]]

## The Hill Climber 
Hill Climb on everything at once

In [15]:
def HillClimber(BestScore, Students, Courses, Triplets, TripletBlocks, FixedX, FixedY, FixedZ, XSet, YSet, ZSet, FixedNum):
    
    # Optimize assignment of triplets to learning groups, and students to learning groups
    solver = pywraplp.Solver('SGS', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    start_time = time.time()

    LearningGroups = [1,2,3]
    Blocks = ['A','B','C','D','E','F','G','H']

    # define boolean variables
    x = {}
    for t in Triplets:
        for l in LearningGroups:
            x[t,l] = solver.IntVar(0,1, 'x[%d,%d]' % (t,l))

    y = {}
    for i in Students:
        for l in LearningGroups:
            y[i,l] = solver.IntVar(0,1, 'y[%d,%d]' % (i,l))

    z = {}
    for i in Students:
        for t in Triplets:
            for l in LearningGroups:
                z[i,t,l] = solver.IntVar(0,1, 'z[%d,%d,%d]' % (i,t,l)) 
        
        
    # CONSTRAINT 0: Set the x[t,l], y[i,l], and z[i,t,l] variables that are fixed from previous iterations.
    
    for a in range(len(FixedX)):
        t = FixedX[a][0]
        l = FixedX[a][1]
        solver.Add(x[t,l] == 1)
        
    for a in range(len(FixedY)):
        i = FixedY[a][0]
        l = FixedY[a][1]
        solver.Add(y[i,l] == 1)
        
    for a in range(len(FixedZ)):
        i = FixedZ[a][0]
        t = FixedZ[a][1]
        l = FixedZ[a][2]
        solver.Add(z[i,t,l] == 1)


    # CONSTRAINT 1: Ensure each triplet is assigned to at most ONE learning group, unless otherwise specified
    for t in Triplets:
        if t in CombinedLG:
            solver.Add(sum(x[t,l] for l in LearningGroups) <= 4)
        else:
            solver.Add(sum(x[t,l] for l in LearningGroups) <= 1)


    # CONSTRAINT 2: Ensure each student is assigned to at most ONE learning group
    for i in Students:
        solver.Add(sum(y[i,l] for l in LearningGroups) <= 1)
        
        
    # CONSTRAINT 2.0: Fix all the assignments, except for a fixed number
    # Do not shuffle the assignments that are fixed from previous iterations
    NewXSet = []
    for xx in XSet:
        if not xx in FixedX:
            NewXSet.append(xx)
    shuffle(NewXSet)
    for i in range(FixedNum, len(NewXSet)):
        t = XSet[i][0]
        l = XSet[i][1]
        solver.Add(x[t,l] == 1)     


    # CONSTRAINT 3: No student can take a course unless it is available to students in their learning group
    for i in Students:
        for t in Triplets:
            for l in LearningGroups:
                solver.Add(z[i,t,l] <= y[i,l])
                solver.Add(z[i,t,l] <= x[t,l])


    # CONSTRAINT 4: No student can take more than one course in a block
    for i in Students:
        for r in range(8):
            solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletBlocks[r]) <= 1)


    # CONSTRAINT 5: NO student can take the same course twice       
    for i in Students:
        for j in Courses:
            solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) <= 1)


    # CONSTRAINT 6: Assign students only the course they requested(i.e. don't assign course j to a student i if P[i,j]=0)      
    for i in Students:
        for j in Courses:
            if P[i,j]==0:
                solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) == 0)


    # CONSTRAINT 7: No course section can exceed its prescribed capacity
    for t in Triplets:
        cap = TripletCap[t]
        solver.Add(sum(z[i,t,l] for l in LearningGroups for i in Students) <= cap)


    # CONSTRAINT 8: For the 807 must courses, the student must be assigned to a section of
    # that course, though the block/section is NOT fixed.
    for i in Students:
        for j in Courses:
            if P[i,j]==10:
                solver.Add(sum(z[i,t,l] for l in LearningGroups for t in TripletIDs[j]) == 1)
                

    # CONSTRAINT 9: Assign at most 110 students to each learning group       
    for l in LearningGroups:
        solver.Add(sum(y[i,l] for i in Students) <= 110) 


                    
    solver.Maximize(solver.Sum(P[i,TripletList[t][0]]*z[i,t,l] 
                               for i in Students for t in Triplets for l in LearningGroups))
    sol = solver.Solve()

    
       
    ObjectiveValue = round(solver.Objective().Value())
    
    
    if ObjectiveValue > BestScore:
        BestScore = ObjectiveValue
              
        XSet=[]
        for t in Triplets:
            for l in LearningGroups:
                if x[t,l].solution_value()==1:
                    XSet.append([t,l])       
                    
        YSet=[]
        for i in Students:
            for l in LearningGroups:
                if y[i,l].solution_value()==1:
                    YSet.append([i,l])
                    
        ZSet=[]
        for i in Students:
            for t in Triplets:
                for l in LearningGroups:
                    if z[i,t,l].solution_value()==1:
                        ZSet.append([i,t,l])
                                        
                    

    return [BestScore, XSet, YSet, ZSet] 

In [16]:
# Find all the Triplets that belong to the same course, and to the same block

Blocks = ['A','B','C','D','E','F','G','H']

TripletIDs = [ [] for r in range(m)]
TripletBlocks = [ [] for r in range(8)]

for t in range(len(TripletList)):
    ID = TripletList[t][0]
    Block = TripletList[t][2]
    TripletIDs[ID].append(t)
    for r in range(8):
        if Blocks[r]==Block:
            TripletBlocks[r].append(t)

In [17]:
# Start the climb at the Progressive Assignment solution output
HighMedLowXSet = [[0, 2], [1, 3], [2, 3], [3, 1], [4, 1], [5, 3], [6, 2], [7, 1], [8, 2], [9, 2], [10, 1], [10, 2], [11, 1], [12, 1], [13, 3], [14, 1], [15, 3], [16, 3], [17, 2], [18, 1], [19, 3], [20, 1], [21, 2], [22, 2], [23, 2], [24, 2], [25, 1], [26, 3], [27, 1], [28, 1], [29, 1], [30, 3], [31, 1], [32, 2], [33, 1], [34, 3], [35, 2], [36, 2], [37, 1], [38, 1], [38, 2], [38, 3], [39, 1], [39, 2], [39, 3], [40, 1], [40, 2], [40, 3], [41, 1], [41, 2], [42, 1], [42, 2], [43, 1], [43, 2], [43, 3], [44, 1], [44, 2], [44, 3], [45, 1], [45, 2], [45, 3], [46, 1], [46, 2], [46, 3], [47, 1], [47, 2], [47, 3], [48, 1], [48, 2], [48, 3], [49, 1], [49, 2], [49, 3], [50, 1], [50, 2], [53, 1], [53, 2], [53, 3], [54, 1], [54, 2], [54, 3], [55, 3], [56, 1], [57, 3], [58, 1], [59, 1], [60, 3], [61, 3], [62, 2], [63, 1], [64, 3], [65, 1], [66, 1], [67, 2], [68, 3], [69, 1], [70, 3], [71, 3], [72, 2], [73, 3], [74, 1], [75, 2], [76, 3], [77, 3], [78, 2], [79, 2], [80, 3], [81, 2], [82, 1], [83, 3], [84, 2], [85, 1], [86, 2], [87, 3], [88, 1], [88, 2], [88, 3], [89, 3], [90, 2], [91, 2], [92, 3], [93, 2], [94, 3], [95, 3], [96, 2], [97, 3], [98, 2], [99, 3], [100, 1], [100, 2], [100, 3], [101, 1], [102, 1], [103, 3], [104, 1], [105, 3], [106, 2], [106, 3], [107, 1], [108, 1], [108, 3], [109, 3], [110, 3], [111, 1], [112, 2], [113, 1], [114, 2], [115, 3], [116, 2], [117, 1], [118, 1], [118, 2], [118, 3], [119, 3], [120, 2], [121, 1], [122, 1], [123, 3], [124, 1], [124, 2], [124, 3], [125, 3], [126, 1], [127, 3], [128, 3], [129, 2], [130, 1], [131, 2], [132, 2], [133, 3], [134, 1], [135, 2], [136, 1], [137, 2], [138, 1], [139, 3], [140, 1], [141, 3], [142, 2], [143, 2], [144, 2], [145, 1], [146, 2], [147, 3], [148, 2], [149, 1], [150, 3], [151, 3], [152, 1], [152, 2], [152, 3], [153, 2], [154, 1], [155, 1], [156, 3], [157, 3], [174, 1], [175, 3], [176, 2], [177, 3], [178, 2], [179, 1], [180, 3], [181, 1], [181, 2], [181, 3], [182, 1], [182, 2], [182, 3], [183, 1], [183, 2], [183, 3], [184, 1], [184, 2], [184, 3], [185, 1], [185, 2], [185, 3], [186, 1], [186, 2], [186, 3], [187, 1], [187, 2], [187, 3], [188, 1], [188, 2], [188, 3], [189, 1], [190, 3], [191, 1], [191, 3], [192, 1], [193, 2], [194, 1], [195, 3]]
HighMedLowYSet = [[0, 3], [1, 3], [2, 3], [3, 1], [4, 3], [5, 3], [6, 1], [7, 1], [8, 3], [9, 3], [10, 3], [11, 1], [12, 3], [13, 1], [14, 1], [15, 3], [16, 1], [17, 3], [18, 1], [19, 1], [20, 3], [21, 3], [22, 3], [23, 3], [24, 3], [25, 3], [26, 2], [27, 1], [28, 2], [29, 2], [30, 3], [31, 2], [32, 1], [33, 3], [34, 3], [35, 1], [36, 2], [37, 3], [38, 2], [39, 1], [40, 1], [41, 3], [42, 3], [43, 3], [44, 2], [45, 3], [46, 3], [47, 3], [48, 3], [49, 2], [50, 2], [51, 1], [52, 2], [53, 2], [54, 1], [55, 2], [56, 3], [57, 1], [58, 3], [59, 2], [60, 2], [61, 2], [62, 2], [63, 2], [64, 2], [65, 2], [66, 1], [67, 1], [68, 1], [69, 3], [70, 3], [71, 2], [72, 3], [73, 1], [74, 3], [75, 1], [76, 1], [77, 3], [78, 3], [79, 3], [80, 1], [81, 3], [82, 1], [83, 1], [84, 1], [85, 3], [86, 1], [87, 3], [88, 1], [89, 2], [90, 3], [91, 3], [92, 3], [93, 3], [94, 3], [95, 3], [96, 3], [97, 3], [98, 1], [99, 1], [100, 1], [101, 3], [102, 3], [103, 3], [104, 3], [105, 3], [106, 1], [107, 2], [108, 2], [109, 3], [110, 3], [111, 2], [112, 1], [113, 2], [114, 2], [115, 3], [116, 3], [117, 2], [118, 1], [119, 1], [120, 1], [121, 1], [122, 1], [123, 1], [124, 2], [125, 1], [126, 3], [127, 3], [128, 1], [129, 3], [130, 1], [131, 2], [132, 1], [133, 2], [134, 1], [135, 2], [136, 3], [137, 2], [138, 3], [139, 3], [140, 2], [141, 2], [142, 2], [143, 2], [144, 2], [145, 2], [146, 3], [147, 3], [148, 2], [149, 3], [150, 3], [151, 3], [152, 1], [153, 3], [154, 1], [155, 3], [156, 1], [157, 3], [158, 1], [159, 1], [160, 2], [161, 3], [162, 1], [163, 1], [164, 1], [165, 2], [166, 2], [167, 3], [168, 2], [169, 3], [170, 1], [171, 2], [172, 3], [173, 1], [174, 1], [175, 3], [176, 1], [177, 2], [178, 1], [179, 2], [180, 1], [181, 3], [182, 1], [183, 3], [184, 3], [185, 1], [186, 1], [187, 1], [188, 3], [189, 1], [190, 1], [191, 1], [192, 2], [193, 1], [194, 2], [195, 3], [196, 2], [197, 3], [198, 1], [199, 3], [200, 1], [201, 1], [202, 1], [203, 2], [204, 2], [205, 3], [206, 2], [207, 2], [208, 2], [209, 1], [210, 1], [211, 3], [212, 3], [213, 1], [214, 3], [215, 3], [216, 2], [217, 3], [218, 2], [219, 2], [220, 1], [221, 1], [222, 2], [223, 3], [224, 2], [225, 3], [226, 3], [227, 2], [228, 2], [229, 2], [230, 2], [231, 2], [232, 2], [233, 1], [234, 3], [235, 1], [236, 1], [237, 2], [238, 1], [239, 1], [240, 3], [241, 2], [242, 1], [243, 1], [244, 1], [245, 2], [246, 1], [247, 1], [248, 3], [249, 3], [250, 3], [251, 3], [252, 1], [253, 1], [254, 2], [255, 1], [256, 3], [257, 1], [258, 1], [259, 1], [260, 2], [261, 2], [262, 1], [263, 2], [264, 2], [265, 2], [266, 1], [267, 1], [268, 1], [269, 2], [270, 2], [271, 2], [272, 2], [273, 2], [274, 1], [275, 2], [276, 2], [277, 3], [278, 2], [279, 2], [280, 2], [281, 2], [282, 3], [283, 1], [284, 3], [285, 2], [286, 1], [287, 1], [288, 1], [289, 3], [290, 2], [291, 2], [292, 2], [293, 2], [294, 2], [295, 2], [296, 1], [297, 2], [298, 2], [299, 2], [300, 1], [301, 1], [302, 1], [303, 2], [304, 3], [305, 1], [306, 2], [307, 3], [308, 3], [309, 3], [310, 2], [311, 3], [312, 1], [313, 1], [314, 1], [315, 1], [316, 2], [317, 2], [318, 2], [319, 2], [320, 2], [321, 2], [322, 2], [323, 2], [324, 3], [325, 2], [326, 2], [327, 3]]
HighMedLowZSet = [[0, 1, 3], [0, 54, 3], [0, 68, 3], [0, 103, 3], [0, 188, 3], [1, 1, 3], [1, 54, 3], [1, 68, 3], [1, 103, 3], [1, 127, 3], [2, 57, 3], [2, 70, 3], [2, 76, 3], [2, 109, 3], [2, 147, 3], [2, 180, 3], [3, 7, 1], [3, 14, 1], [3, 25, 1], [3, 74, 1], [3, 111, 1], [3, 145, 1], [4, 61, 3], [4, 70, 3], [4, 76, 3], [4, 87, 3], [4, 110, 3], [5, 71, 3], [5, 97, 3], [5, 109, 3], [5, 119, 3], [5, 127, 3], [5, 177, 3], [5, 183, 3], [6, 12, 1], [6, 27, 1], [6, 65, 1], [6, 108, 1], [6, 174, 1], [7, 7, 1], [7, 48, 1], [7, 65, 1], [7, 102, 1], [7, 111, 1], [7, 189, 1], [8, 30, 3], [8, 71, 3], [8, 109, 3], [8, 119, 3], [8, 157, 3], [9, 13, 3], [9, 26, 3], [9, 44, 3], [9, 73, 3], [9, 125, 3], [9, 133, 3], [9, 157, 3], [10, 1, 3], [10, 16, 3], [10, 68, 3], [10, 99, 3], [10, 177, 3], [11, 18, 1], [11, 27, 1], [11, 54, 1], [11, 66, 1], [11, 108, 1], [11, 192, 1], [12, 13, 3], [12, 26, 3], [12, 71, 3], [12, 119, 3], [12, 125, 3], [12, 147, 3], [13, 12, 1], [13, 14, 1], [13, 29, 1], [13, 65, 1], [13, 134, 1], [13, 183, 1], [14, 12, 1], [14, 18, 1], [14, 65, 1], [14, 102, 1], [14, 134, 1], [14, 174, 1], [15, 16, 3], [15, 68, 3], [15, 180, 3], [16, 7, 1], [16, 29, 1], [16, 45, 1], [16, 65, 1], [16, 102, 1], [16, 136, 1], [16, 174, 1], [17, 15, 3], [17, 57, 3], [17, 70, 3], [17, 109, 3], [17, 147, 3], [18, 18, 1], [18, 65, 1], [18, 102, 1], [18, 155, 1], [18, 181, 1], [19, 18, 1], [19, 27, 1], [19, 45, 1], [19, 66, 1], [19, 136, 1], [19, 174, 1], [20, 70, 3], [20, 99, 3], [20, 109, 3], [20, 152, 3], [20, 180, 3], [21, 5, 3], [21, 13, 3], [21, 26, 3], [21, 73, 3], [21, 127, 3], [21, 180, 3], [22, 13, 3], [22, 71, 3], [22, 76, 3], [22, 97, 3], [22, 119, 3], [22, 147, 3], [23, 57, 3], [23, 70, 3], [23, 97, 3], [23, 109, 3], [23, 147, 3], [23, 187, 3], [24, 15, 3], [24, 61, 3], [24, 71, 3], [24, 92, 3], [24, 127, 3], [25, 1, 3], [25, 15, 3], [25, 61, 3], [25, 70, 3], [25, 110, 3], [25, 184, 3], [26, 0, 2], [26, 62, 2], [26, 72, 2], [26, 148, 2], [26, 184, 2], [27, 12, 1], [27, 29, 1], [27, 69, 1], [27, 149, 1], [27, 192, 1], [28, 6, 2], [28, 21, 2], [28, 32, 2], [28, 81, 2], [28, 132, 2], [28, 142, 2], [28, 178, 2], [29, 24, 2], [29, 62, 2], [29, 86, 2], [29, 114, 2], [29, 120, 2], [29, 144, 2], [29, 176, 2], [30, 16, 3], [30, 26, 3], [30, 73, 3], [30, 157, 3], [30, 183, 3], [31, 22, 2], [31, 47, 2], [31, 81, 2], [31, 91, 2], [31, 116, 2], [31, 129, 2], [31, 142, 2], [32, 20, 1], [32, 33, 1], [32, 82, 1], [32, 126, 1], [32, 130, 1], [32, 138, 1], [32, 154, 1], [33, 15, 3], [33, 61, 3], [33, 71, 3], [33, 147, 3], [33, 180, 3], [34, 61, 3], [34, 80, 3], [34, 99, 3], [34, 115, 3], [34, 119, 3], [34, 128, 3], [34, 139, 3], [34, 191, 3], [35, 20, 1], [35, 39, 1], [35, 47, 1], [35, 85, 1], [35, 113, 1], [35, 118, 1], [35, 140, 1], [36, 24, 2], [36, 46, 2], [36, 81, 2], [36, 114, 2], [36, 120, 2], [36, 142, 2], [36, 178, 2], [37, 34, 3], [37, 55, 3], [37, 80, 3], [37, 109, 3], [37, 125, 3], [37, 139, 3], [38, 17, 2], [38, 24, 2], [38, 46, 2], [38, 86, 2], [38, 91, 2], [38, 116, 2], [38, 132, 2], [38, 144, 2], [39, 39, 1], [39, 56, 1], [39, 85, 1], [39, 102, 1], [39, 113, 1], [39, 140, 1], [39, 154, 1], [40, 56, 1], [40, 82, 1], [40, 113, 1], [40, 126, 1], [40, 138, 1], [40, 154, 1], [41, 19, 3], [41, 83, 3], [41, 89, 3], [41, 115, 3], [41, 128, 3], [41, 141, 3], [41, 188, 3], [42, 43, 3], [42, 83, 3], [42, 89, 3], [42, 115, 3], [42, 123, 3], [42, 128, 3], [42, 141, 3], [43, 34, 3], [43, 55, 3], [43, 80, 3], [43, 110, 3], [43, 123, 3], [43, 125, 3], [43, 139, 3], [44, 6, 2], [44, 22, 2], [44, 36, 2], [44, 81, 2], [44, 93, 2], [44, 106, 2], [44, 142, 2], [45, 13, 3], [45, 73, 3], [45, 119, 3], [45, 125, 3], [45, 147, 3], [46, 5, 3], [46, 13, 3], [46, 26, 3], [46, 73, 3], [46, 88, 3], [46, 133, 3], [47, 5, 3], [47, 80, 3], [47, 106, 3], [47, 115, 3], [47, 128, 3], [47, 139, 3], [48, 34, 3], [48, 53, 3], [48, 55, 3], [48, 80, 3], [48, 89, 3], [48, 139, 3], [48, 181, 3], [49, 23, 2], [49, 36, 2], [49, 45, 2], [49, 53, 2], [49, 81, 2], [49, 91, 2], [49, 132, 2], [49, 142, 2], [50, 6, 2], [50, 22, 2], [50, 81, 2], [50, 106, 2], [50, 116, 2], [50, 143, 2], [51, 33, 1], [51, 56, 1], [51, 82, 1], [51, 138, 1], [51, 179, 1], [51, 191, 1], [52, 22, 2], [52, 36, 2], [52, 40, 2], [52, 62, 2], [52, 86, 2], [52, 137, 2], [52, 153, 2], [53, 22, 2], [53, 36, 2], [53, 47, 2], [53, 62, 2], [53, 81, 2], [53, 132, 2], [53, 143, 2], [53, 153, 2], [54, 7, 1], [54, 14, 1], [54, 25, 1], [54, 74, 1], [54, 111, 1], [54, 184, 1], [55, 24, 2], [55, 36, 2], [55, 62, 2], [55, 86, 2], [55, 91, 2], [55, 131, 2], [55, 144, 2], [56, 34, 3], [56, 55, 3], [56, 80, 3], [56, 89, 3], [56, 100, 3], [56, 110, 3], [56, 139, 3], [57, 12, 1], [57, 27, 1], [57, 66, 1], [57, 108, 1], [57, 174, 1], [58, 45, 3], [58, 46, 3], [58, 83, 3], [58, 89, 3], [58, 115, 3], [58, 128, 3], [58, 141, 3], [59, 21, 2], [59, 32, 2], [59, 62, 2], [59, 86, 2], [59, 137, 2], [59, 176, 2], [60, 23, 2], [60, 44, 2], [60, 47, 2], [60, 81, 2], [60, 93, 2], [60, 116, 2], [60, 131, 2], [60, 143, 2], [61, 23, 2], [61, 35, 2], [61, 47, 2], [61, 81, 2], [61, 93, 2], [61, 132, 2], [61, 143, 2], [62, 6, 2], [62, 17, 2], [62, 24, 2], [62, 79, 2], [62, 93, 2], [62, 114, 2], [62, 129, 2], [62, 143, 2], [63, 17, 2], [63, 24, 2], [63, 36, 2], [63, 62, 2], [63, 79, 2], [63, 112, 2], [63, 143, 2], [63, 153, 2], [64, 23, 2], [64, 35, 2], [64, 47, 2], [64, 79, 2], [64, 91, 2], [64, 131, 2], [64, 143, 2], [65, 22, 2], [65, 36, 2], [65, 62, 2], [65, 86, 2], [65, 96, 2], [65, 106, 2], [65, 132, 2], [65, 144, 2], [66, 20, 1], [66, 85, 1], [66, 113, 1], [66, 122, 1], [66, 126, 1], [66, 130, 1], [66, 140, 1], [66, 154, 1], [67, 7, 1], [67, 20, 1], [67, 53, 1], [67, 85, 1], [67, 113, 1], [67, 140, 1], [67, 154, 1], [68, 27, 1], [68, 66, 1], [68, 136, 1], [68, 174, 1], [68, 188, 1], [69, 16, 3], [69, 26, 3], [69, 73, 3], [69, 127, 3], [69, 133, 3], [69, 190, 3], [70, 1, 3], [70, 16, 3], [70, 73, 3], [70, 99, 3], [70, 103, 3], [71, 79, 2], [71, 96, 2], [71, 106, 2], [71, 116, 2], [71, 131, 2], [71, 143, 2], [71, 176, 2], [72, 5, 3], [72, 13, 3], [72, 26, 3], [72, 71, 3], [72, 127, 3], [72, 180, 3], [73, 20, 1], [73, 85, 1], [73, 113, 1], [73, 118, 1], [73, 130, 1], [73, 140, 1], [73, 179, 1], [74, 5, 3], [74, 26, 3], [74, 61, 3], [74, 71, 3], [74, 119, 3], [74, 127, 3], [75, 14, 1], [75, 25, 1], [75, 69, 1], [75, 134, 1], [75, 155, 1], [76, 12, 1], [76, 18, 1], [76, 66, 1], [76, 108, 1], [76, 192, 1], [77, 15, 3], [77, 68, 3], [77, 110, 3], [77, 123, 3], [77, 157, 3], [78, 5, 3], [78, 13, 3], [78, 26, 3], [78, 71, 3], [78, 119, 3], [78, 127, 3], [78, 180, 3], [79, 1, 3], [79, 71, 3], [79, 92, 3], [79, 99, 3], [79, 175, 3], [80, 7, 1], [80, 12, 1], [80, 29, 1], [80, 65, 1], [80, 174, 1], [81, 16, 3], [81, 26, 3], [81, 61, 3], [81, 68, 3], [82, 14, 1], [82, 25, 1], [82, 44, 1], [82, 74, 1], [82, 134, 1], [83, 18, 1], [83, 27, 1], [83, 48, 1], [83, 65, 1], [83, 149, 1], [84, 7, 1], [84, 25, 1], [84, 49, 1], [84, 69, 1], [84, 145, 1], [84, 155, 1], [85, 16, 3], [85, 49, 3], [85, 68, 3], [85, 103, 3], [85, 124, 3], [86, 18, 1], [86, 29, 1], [86, 41, 1], [86, 49, 1], [86, 66, 1], [86, 136, 1], [87, 1, 3], [87, 71, 3], [87, 92, 3], [87, 97, 3], [87, 109, 3], [87, 175, 3], [88, 12, 1], [88, 27, 1], [88, 65, 1], [88, 111, 1], [88, 174, 1], [89, 48, 2], [89, 72, 2], [89, 146, 2], [89, 148, 2], [89, 186, 2], [90, 13, 3], [90, 49, 3], [90, 73, 3], [90, 92, 3], [90, 110, 3], [90, 125, 3], [91, 15, 3], [91, 40, 3], [91, 70, 3], [91, 110, 3], [91, 119, 3], [91, 156, 3], [92, 57, 3], [92, 70, 3], [92, 76, 3], [92, 99, 3], [92, 109, 3], [92, 125, 3], [93, 19, 3], [93, 46, 3], [93, 83, 3], [93, 89, 3], [93, 115, 3], [93, 128, 3], [93, 141, 3], [94, 13, 3], [94, 71, 3], [94, 119, 3], [94, 125, 3], [94, 147, 3], [94, 177, 3], [95, 15, 3], [95, 30, 3], [95, 61, 3], [95, 71, 3], [95, 119, 3], [95, 127, 3], [96, 13, 3], [96, 26, 3], [96, 71, 3], [96, 125, 3], [96, 133, 3], [96, 157, 3], [97, 1, 3], [97, 30, 3], [97, 71, 3], [97, 109, 3], [97, 127, 3], [97, 187, 3], [98, 33, 1], [98, 56, 1], [98, 82, 1], [98, 126, 1], [98, 138, 1], [98, 154, 1], [99, 12, 1], [99, 18, 1], [99, 27, 1], [99, 48, 1], [99, 69, 1], [99, 102, 1], [99, 134, 1], [100, 27, 1], [100, 49, 1], [100, 63, 1], [100, 69, 1], [100, 102, 1], [100, 136, 1], [100, 174, 1], [101, 13, 3], [101, 70, 3], [101, 99, 3], [101, 110, 3], [101, 125, 3], [102, 15, 3], [102, 30, 3], [102, 61, 3], [102, 70, 3], [102, 110, 3], [102, 119, 3], [102, 127, 3], [103, 1, 3], [103, 15, 3], [103, 54, 3], [103, 61, 3], [103, 73, 3], [104, 5, 3], [104, 13, 3], [104, 26, 3], [104, 49, 3], [104, 68, 3], [105, 34, 3], [105, 55, 3], [105, 80, 3], [105, 89, 3], [105, 109, 3], [105, 139, 3], [105, 175, 3], [106, 18, 1], [106, 29, 1], [106, 49, 1], [106, 66, 1], [106, 136, 1], [106, 192, 1], [107, 0, 2], [107, 72, 2], [107, 148, 2], [107, 152, 2], [108, 22, 2], [108, 86, 2], [108, 114, 2], [108, 137, 2], [108, 153, 2], [109, 15, 3], [109, 30, 3], [109, 70, 3], [109, 110, 3], [109, 123, 3], [109, 127, 3], [110, 68, 3], [110, 103, 3], [110, 110, 3], [110, 157, 3], [111, 0, 2], [111, 6, 2], [111, 72, 2], [111, 146, 2], [112, 20, 1], [112, 33, 1], [112, 82, 1], [112, 130, 1], [112, 138, 1], [112, 152, 1], [112, 154, 1], [113, 46, 2], [113, 78, 2], [113, 86, 2], [113, 93, 2], [113, 116, 2], [113, 137, 2], [114, 17, 2], [114, 24, 2], [114, 46, 2], [114, 79, 2], [114, 114, 2], [114, 120, 2], [114, 142, 2], [115, 55, 3], [115, 83, 3], [115, 89, 3], [115, 103, 3], [115, 110, 3], [115, 115, 3], [115, 141, 3], [116, 1, 3], [116, 73, 3], [116, 109, 3], [116, 147, 3], [116, 152, 3], [116, 183, 3], [117, 17, 2], [117, 24, 2], [117, 35, 2], [117, 46, 2], [117, 84, 2], [117, 91, 2], [117, 132, 2], [117, 144, 2], [118, 33, 1], [118, 56, 1], [118, 82, 1], [118, 126, 1], [118, 138, 1], [118, 154, 1], [119, 29, 1], [119, 44, 1], [119, 49, 1], [119, 66, 1], [119, 136, 1], [119, 174, 1], [120, 33, 1], [120, 50, 1], [120, 85, 1], [120, 130, 1], [120, 140, 1], [120, 154, 1], [121, 20, 1], [121, 33, 1], [121, 82, 1], [121, 126, 1], [121, 130, 1], [121, 138, 1], [121, 154, 1], [122, 33, 1], [122, 56, 1], [122, 85, 1], [122, 118, 1], [122, 126, 1], [122, 140, 1], [122, 191, 1], [123, 14, 1], [123, 25, 1], [123, 74, 1], [123, 134, 1], [123, 145, 1], [124, 0, 2], [124, 62, 2], [124, 72, 2], [124, 146, 2], [125, 14, 1], [125, 25, 1], [125, 42, 1], [125, 74, 1], [125, 134, 1], [126, 1, 3], [126, 15, 3], [126, 30, 3], [126, 49, 3], [126, 70, 3], [126, 110, 3], [126, 124, 3], [127, 1, 3], [127, 13, 3], [127, 70, 3], [127, 110, 3], [127, 125, 3], [127, 147, 3], [128, 18, 1], [128, 27, 1], [128, 66, 1], [128, 134, 1], [128, 183, 1], [129, 55, 3], [129, 83, 3], [129, 109, 3], [129, 115, 3], [129, 141, 3], [129, 190, 3], [130, 20, 1], [130, 33, 1], [130, 85, 1], [130, 118, 1], [130, 130, 1], [130, 140, 1], [131, 21, 2], [131, 62, 2], [131, 81, 2], [131, 116, 2], [131, 142, 2], [131, 153, 2], [132, 27, 1], [132, 44, 1], [132, 69, 1], [132, 111, 1], [132, 149, 1], [132, 155, 1], [132, 174, 1], [133, 38, 2], [133, 78, 2], [133, 81, 2], [133, 88, 2], [133, 91, 2], [133, 116, 2], [133, 142, 2], [134, 56, 1], [134, 82, 1], [134, 100, 1], [134, 113, 1], [134, 126, 1], [134, 138, 1], [134, 154, 1], [135, 0, 2], [135, 6, 2], [135, 72, 2], [135, 88, 2], [135, 184, 2], [136, 54, 3], [136, 73, 3], [136, 92, 3], [136, 109, 3], [136, 125, 3], [136, 184, 3], [137, 86, 2], [137, 100, 2], [137, 116, 2], [137, 137, 2], [137, 153, 2], [137, 176, 2], [137, 181, 2], [138, 1, 3], [138, 30, 3], [138, 71, 3], [138, 125, 3], [138, 147, 3], [138, 177, 3], [139, 1, 3], [139, 16, 3], [139, 26, 3], [139, 73, 3], [139, 109, 3], [140, 23, 2], [140, 79, 2], [140, 91, 2], [140, 114, 2], [140, 129, 2], [140, 142, 2], [140, 178, 2], [141, 35, 2], [141, 78, 2], [141, 86, 2], [141, 137, 2], [141, 153, 2], [142, 0, 2], [142, 62, 2], [142, 72, 2], [142, 152, 2], [142, 184, 2], [143, 23, 2], [143, 36, 2], [143, 47, 2], [143, 79, 2], [143, 91, 2], [143, 131, 2], [143, 142, 2], [144, 6, 2], [144, 24, 2], [144, 35, 2], [144, 79, 2], [144, 93, 2], [144, 129, 2], [144, 142, 2], [144, 178, 2], [145, 24, 2], [145, 32, 2], [145, 43, 2], [145, 79, 2], [145, 90, 2], [145, 129, 2], [145, 142, 2], [146, 43, 3], [146, 83, 3], [146, 106, 3], [146, 115, 3], [146, 128, 3], [146, 141, 3], [147, 5, 3], [147, 13, 3], [147, 26, 3], [147, 73, 3], [147, 110, 3], [147, 147, 3], [147, 180, 3], [148, 6, 2], [148, 72, 2], [148, 146, 2], [149, 1, 3], [149, 16, 3], [149, 70, 3], [149, 76, 3], [149, 103, 3], [149, 157, 3], [150, 70, 3], [150, 76, 3], [150, 88, 3], [150, 97, 3], [150, 147, 3], [150, 190, 3], [151, 15, 3], [151, 61, 3], [151, 73, 3], [151, 99, 3], [151, 152, 3], [151, 180, 3], [152, 12, 1], [152, 18, 1], [152, 27, 1], [152, 48, 1], [152, 66, 1], [153, 1, 3], [153, 70, 3], [153, 87, 3], [153, 97, 3], [153, 109, 3], [153, 185, 3], [154, 54, 1], [154, 69, 1], [154, 126, 1], [154, 145, 1], [154, 149, 1], [154, 192, 1], [155, 15, 3], [155, 61, 3], [155, 70, 3], [155, 88, 3], [155, 147, 3], [156, 7, 1], [156, 14, 1], [156, 25, 1], [156, 69, 1], [156, 155, 1], [157, 5, 3], [157, 26, 3], [157, 61, 3], [157, 73, 3], [157, 119, 3], [157, 147, 3], [157, 180, 3], [158, 7, 1], [158, 14, 1], [158, 25, 1], [158, 74, 1], [158, 111, 1], [158, 145, 1], [158, 149, 1], [159, 18, 1], [159, 27, 1], [159, 69, 1], [159, 136, 1], [159, 174, 1], [160, 0, 2], [160, 72, 2], [160, 146, 2], [160, 148, 2], [161, 68, 3], [161, 97, 3], [161, 109, 3], [161, 152, 3], [161, 177, 3], [162, 18, 1], [162, 27, 1], [162, 74, 1], [162, 134, 1], [162, 174, 1], [163, 27, 1], [163, 49, 1], [163, 65, 1], [163, 136, 1], [163, 192, 1], [164, 29, 1], [164, 49, 1], [164, 63, 1], [164, 65, 1], [164, 174, 1], [165, 23, 2], [165, 32, 2], [165, 79, 2], [165, 90, 2], [165, 132, 2], [165, 143, 2], [165, 178, 2], [166, 21, 2], [166, 79, 2], [166, 116, 2], [166, 132, 2], [166, 143, 2], [166, 153, 2], [166, 178, 2], [167, 57, 3], [167, 70, 3], [167, 103, 3], [167, 109, 3], [167, 127, 3], [167, 180, 3], [168, 0, 2], [168, 6, 2], [168, 36, 2], [168, 79, 2], [168, 106, 2], [168, 143, 2], [169, 68, 3], [169, 99, 3], [169, 156, 3], [170, 18, 1], [170, 29, 1], [170, 66, 1], [170, 136, 1], [170, 189, 1], [171, 22, 2], [171, 46, 2], [171, 84, 2], [171, 91, 2], [171, 114, 2], [171, 129, 2], [171, 144, 2], [172, 5, 3], [172, 26, 3], [172, 73, 3], [172, 88, 3], [172, 109, 3], [172, 147, 3], [172, 180, 3], [173, 7, 1], [173, 14, 1], [173, 25, 1], [173, 69, 1], [173, 111, 1], [173, 155, 1], [173, 184, 1], [174, 27, 1], [174, 49, 1], [174, 69, 1], [174, 136, 1], [174, 174, 1], [175, 5, 3], [175, 19, 3], [175, 34, 3], [175, 83, 3], [175, 89, 3], [175, 128, 3], [175, 141, 3], [176, 12, 1], [176, 14, 1], [176, 25, 1], [176, 69, 1], [176, 111, 1], [176, 149, 1], [177, 17, 2], [177, 23, 2], [177, 36, 2], [177, 86, 2], [177, 93, 2], [177, 106, 2], [177, 132, 2], [177, 144, 2], [178, 20, 1], [178, 82, 1], [178, 113, 1], [178, 130, 1], [178, 138, 1], [178, 154, 1], [178, 179, 1], [179, 17, 2], [179, 24, 2], [179, 32, 2], [179, 46, 2], [179, 86, 2], [179, 90, 2], [179, 132, 2], [179, 144, 2], [180, 33, 1], [180, 56, 1], [180, 85, 1], [180, 118, 1], [180, 140, 1], [181, 15, 3], [181, 30, 3], [181, 61, 3], [181, 70, 3], [181, 110, 3], [181, 184, 3], [182, 14, 1], [182, 25, 1], [182, 63, 1], [182, 65, 1], [182, 145, 1], [183, 5, 3], [183, 73, 3], [183, 88, 3], [183, 109, 3], [183, 147, 3], [184, 1, 3], [184, 15, 3], [184, 61, 3], [184, 73, 3], [184, 147, 3], [184, 180, 3], [185, 12, 1], [185, 18, 1], [185, 25, 1], [185, 69, 1], [186, 14, 1], [186, 25, 1], [186, 54, 1], [186, 69, 1], [186, 126, 1], [186, 184, 1], [187, 7, 1], [187, 82, 1], [187, 113, 1], [187, 130, 1], [187, 138, 1], [187, 154, 1], [188, 1, 3], [188, 61, 3], [188, 70, 3], [188, 97, 3], [188, 110, 3], [188, 147, 3], [189, 29, 1], [189, 69, 1], [189, 136, 1], [189, 174, 1], [190, 18, 1], [190, 25, 1], [190, 65, 1], [190, 108, 1], [190, 189, 1], [191, 18, 1], [191, 27, 1], [191, 74, 1], [191, 102, 1], [191, 111, 1], [191, 134, 1], [192, 10, 2], [192, 72, 2], [192, 146, 2], [193, 54, 1], [193, 65, 1], [193, 102, 1], [193, 111, 1], [193, 149, 1], [193, 183, 1], [193, 189, 1], [194, 50, 2], [194, 84, 2], [194, 91, 2], [194, 106, 2], [194, 114, 2], [194, 129, 2], [194, 144, 2], [195, 34, 3], [195, 55, 3], [195, 57, 3], [195, 80, 3], [195, 89, 3], [195, 100, 3], [195, 139, 3], [196, 22, 2], [196, 36, 2], [196, 40, 2], [196, 62, 2], [196, 79, 2], [196, 120, 2], [196, 129, 2], [196, 143, 2], [197, 5, 3], [197, 13, 3], [197, 26, 3], [197, 73, 3], [197, 110, 3], [197, 147, 3], [197, 180, 3], [198, 12, 1], [198, 18, 1], [198, 27, 1], [198, 48, 1], [198, 65, 1], [198, 108, 1], [198, 149, 1], [199, 19, 3], [199, 34, 3], [199, 83, 3], [199, 128, 3], [199, 141, 3], [200, 33, 1], [200, 56, 1], [200, 82, 1], [200, 100, 1], [200, 126, 1], [200, 138, 1], [200, 154, 1], [201, 29, 1], [201, 42, 1], [201, 65, 1], [201, 136, 1], [201, 192, 1], [202, 56, 1], [202, 85, 1], [202, 113, 1], [202, 126, 1], [202, 140, 1], [202, 154, 1], [203, 54, 2], [203, 72, 2], [203, 148, 2], [203, 184, 2], [204, 23, 2], [204, 36, 2], [204, 79, 2], [204, 91, 2], [204, 112, 2], [204, 143, 2], [204, 178, 2], [205, 19, 3], [205, 80, 3], [205, 89, 3], [205, 115, 3], [205, 128, 3], [205, 139, 3], [206, 6, 2], [206, 24, 2], [206, 36, 2], [206, 81, 2], [206, 96, 2], [206, 131, 2], [206, 143, 2], [207, 6, 2], [207, 22, 2], [207, 47, 2], [207, 84, 2], [207, 116, 2], [207, 137, 2], [208, 17, 2], [208, 23, 2], [208, 36, 2], [208, 78, 2], [208, 84, 2], [208, 120, 2], [208, 132, 2], [208, 144, 2], [209, 20, 1], [209, 85, 1], [209, 88, 1], [209, 113, 1], [209, 130, 1], [209, 140, 1], [209, 154, 1], [210, 56, 1], [210, 82, 1], [210, 100, 1], [210, 113, 1], [210, 138, 1], [210, 154, 1], [211, 19, 3], [211, 34, 3], [211, 80, 3], [211, 89, 3], [211, 127, 3], [211, 128, 3], [211, 139, 3], [211, 175, 3], [212, 5, 3], [212, 83, 3], [212, 89, 3], [212, 115, 3], [212, 128, 3], [212, 141, 3], [213, 14, 1], [213, 25, 1], [213, 74, 1], [213, 111, 1], [213, 126, 1], [213, 145, 1], [214, 1, 3], [214, 71, 3], [214, 97, 3], [214, 103, 3], [214, 109, 3], [214, 147, 3], [215, 68, 3], [215, 103, 3], [215, 110, 3], [215, 156, 3], [215, 157, 3], [216, 24, 2], [216, 84, 2], [216, 114, 2], [216, 120, 2], [216, 137, 2], [216, 178, 2], [217, 5, 3], [217, 19, 3], [217, 80, 3], [217, 89, 3], [217, 115, 3], [217, 139, 3], [217, 186, 3], [218, 84, 2], [218, 116, 2], [218, 137, 2], [218, 153, 2], [218, 176, 2], [218, 178, 2], [219, 32, 2], [219, 84, 2], [219, 90, 2], [219, 112, 2], [219, 137, 2], [219, 178, 2], [220, 20, 1], [220, 85, 1], [220, 113, 1], [220, 118, 1], [220, 130, 1], [220, 140, 1], [220, 179, 1], [221, 56, 1], [221, 82, 1], [221, 113, 1], [221, 126, 1], [221, 138, 1], [221, 154, 1], [221, 179, 1], [222, 23, 2], [222, 86, 2], [222, 91, 2], [222, 106, 2], [222, 116, 2], [222, 131, 2], [222, 144, 2], [223, 19, 3], [223, 34, 3], [223, 80, 3], [223, 89, 3], [223, 128, 3], [223, 139, 3], [223, 175, 3], [224, 17, 2], [224, 24, 2], [224, 36, 2], [224, 43, 2], [224, 86, 2], [224, 91, 2], [224, 132, 2], [224, 144, 2], [225, 19, 3], [225, 34, 3], [225, 83, 3], [225, 99, 3], [225, 110, 3], [225, 141, 3], [226, 34, 3], [226, 55, 3], [226, 57, 3], [226, 80, 3], [226, 89, 3], [226, 139, 3], [226, 181, 3], [227, 23, 2], [227, 32, 2], [227, 86, 2], [227, 90, 2], [227, 129, 2], [227, 137, 2], [227, 148, 2], [228, 21, 2], [228, 35, 2], [228, 53, 2], [228, 81, 2], [228, 112, 2], [228, 120, 2], [228, 142, 2], [228, 181, 2], [229, 22, 2], [229, 36, 2], [229, 46, 2], [229, 78, 2], [229, 81, 2], [229, 91, 2], [229, 142, 2], [230, 17, 2], [230, 24, 2], [230, 46, 2], [230, 86, 2], [230, 114, 2], [230, 132, 2], [230, 144, 2], [230, 153, 2], [231, 6, 2], [231, 22, 2], [231, 81, 2], [231, 106, 2], [231, 114, 2], [231, 142, 2], [232, 22, 2], [232, 36, 2], [232, 45, 2], [232, 62, 2], [232, 79, 2], [232, 120, 2], [232, 129, 2], [232, 142, 2], [233, 33, 1], [233, 56, 1], [233, 82, 1], [233, 118, 1], [233, 138, 1], [233, 179, 1], [234, 13, 3], [234, 71, 3], [234, 76, 3], [234, 127, 3], [234, 180, 3], [235, 7, 1], [235, 33, 1], [235, 85, 1], [235, 140, 1], [236, 29, 1], [236, 66, 1], [236, 136, 1], [236, 192, 1], [237, 23, 2], [237, 35, 2], [237, 39, 2], [237, 81, 2], [237, 131, 2], [237, 142, 2], [237, 153, 2], [237, 188, 2], [238, 20, 1], [238, 85, 1], [238, 113, 1], [238, 122, 1], [238, 130, 1], [238, 140, 1], [238, 154, 1], [239, 33, 1], [239, 82, 1], [239, 118, 1], [239, 130, 1], [239, 138, 1], [240, 19, 3], [240, 34, 3], [240, 80, 3], [240, 89, 3], [240, 99, 3], [240, 110, 3], [240, 139, 3], [240, 175, 3], [241, 17, 2], [241, 35, 2], [241, 46, 2], [241, 81, 2], [241, 120, 2], [241, 132, 2], [241, 142, 2], [242, 7, 1], [242, 14, 1], [242, 25, 1], [242, 74, 1], [243, 20, 1], [243, 82, 1], [243, 102, 1], [243, 113, 1], [243, 118, 1], [243, 130, 1], [243, 138, 1], [244, 20, 1], [244, 33, 1], [244, 39, 1], [244, 82, 1], [244, 118, 1], [244, 130, 1], [244, 138, 1], [245, 6, 2], [245, 17, 2], [245, 24, 2], [245, 81, 2], [245, 116, 2], [245, 142, 2], [245, 153, 2], [246, 29, 1], [246, 63, 1], [246, 66, 1], [246, 136, 1], [246, 174, 1], [246, 189, 1], [247, 29, 1], [247, 63, 1], [247, 66, 1], [247, 155, 1], [247, 192, 1], [248, 16, 3], [248, 26, 3], [248, 73, 3], [248, 133, 3], [248, 157, 3], [249, 34, 3], [249, 55, 3], [249, 80, 3], [249, 139, 3], [249, 191, 3], [250, 19, 3], [250, 80, 3], [250, 89, 3], [250, 115, 3], [250, 127, 3], [250, 128, 3], [250, 139, 3], [250, 175, 3], [251, 34, 3], [251, 55, 3], [251, 83, 3], [251, 141, 3], [251, 181, 3], [252, 7, 1], [252, 20, 1], [252, 85, 1], [252, 113, 1], [252, 118, 1], [252, 140, 1], [252, 186, 1], [253, 25, 1], [253, 74, 1], [253, 126, 1], [253, 134, 1], [254, 35, 2], [254, 84, 2], [254, 93, 2], [254, 112, 2], [254, 137, 2], [255, 14, 1], [255, 63, 1], [255, 74, 1], [255, 111, 1], [255, 174, 1], [256, 16, 3], [256, 26, 3], [256, 68, 3], [256, 99, 3], [256, 133, 3], [257, 7, 1], [257, 14, 1], [257, 29, 1], [257, 74, 1], [257, 145, 1], [258, 14, 1], [258, 74, 1], [258, 111, 1], [258, 126, 1], [259, 18, 1], [259, 29, 1], [259, 65, 1], [259, 102, 1], [259, 174, 1], [259, 189, 1], [260, 0, 2], [260, 54, 2], [260, 72, 2], [260, 187, 2], [261, 21, 2], [261, 35, 2], [261, 86, 2], [261, 129, 2], [261, 144, 2], [261, 153, 2], [261, 185, 2], [262, 33, 1], [262, 56, 1], [262, 63, 1], [262, 85, 1], [262, 140, 1], [262, 154, 1], [263, 22, 2], [263, 36, 2], [263, 47, 2], [263, 62, 2], [263, 81, 2], [263, 91, 2], [263, 143, 2], [264, 24, 2], [264, 36, 2], [264, 81, 2], [264, 129, 2], [264, 143, 2], [264, 153, 2], [265, 6, 2], [265, 36, 2], [265, 81, 2], [265, 143, 2], [265, 148, 2], [265, 153, 2], [266, 12, 1], [266, 29, 1], [266, 49, 1], [266, 69, 1], [266, 174, 1], [267, 14, 1], [267, 63, 1], [267, 69, 1], [267, 155, 1], [267, 174, 1], [268, 12, 1], [268, 14, 1], [268, 29, 1], [268, 65, 1], [268, 134, 1], [268, 145, 1], [268, 187, 1], [269, 21, 2], [269, 81, 2], [269, 114, 2], [269, 120, 2], [269, 129, 2], [269, 143, 2], [270, 10, 2], [270, 72, 2], [270, 96, 2], [270, 146, 2], [270, 148, 2], [271, 23, 2], [271, 84, 2], [271, 91, 2], [271, 114, 2], [271, 132, 2], [271, 148, 2], [271, 186, 2], [272, 22, 2], [272, 32, 2], [272, 62, 2], [272, 79, 2], [272, 143, 2], [272, 186, 2], [273, 17, 2], [273, 72, 2], [273, 120, 2], [273, 187, 2], [274, 66, 1], [274, 111, 1], [274, 136, 1], [274, 174, 1], [274, 184, 1], [274, 189, 1], [275, 17, 2], [275, 24, 2], [275, 35, 2], [275, 46, 2], [275, 84, 2], [275, 93, 2], [275, 144, 2], [276, 17, 2], [276, 47, 2], [276, 62, 2], [276, 79, 2], [276, 114, 2], [276, 120, 2], [276, 143, 2], [277, 16, 3], [277, 68, 3], [278, 17, 2], [278, 23, 2], [278, 35, 2], [278, 47, 2], [278, 84, 2], [278, 96, 2], [278, 132, 2], [278, 144, 2], [279, 22, 2], [279, 35, 2], [279, 79, 2], [279, 120, 2], [279, 129, 2], [279, 143, 2], [280, 23, 2], [280, 35, 2], [280, 84, 2], [280, 129, 2], [280, 137, 2], [280, 153, 2], [280, 178, 2], [281, 62, 2], [281, 72, 2], [281, 96, 2], [281, 152, 2], [282, 34, 3], [282, 55, 3], [282, 80, 3], [282, 88, 3], [282, 109, 3], [282, 119, 3], [282, 139, 3], [283, 82, 1], [283, 88, 1], [283, 113, 1], [283, 118, 1], [283, 138, 1], [284, 19, 3], [284, 34, 3], [284, 80, 3], [284, 87, 3], [284, 89, 3], [284, 128, 3], [284, 139, 3], [285, 24, 2], [285, 35, 2], [285, 79, 2], [285, 120, 2], [285, 131, 2], [285, 143, 2], [285, 176, 2], [286, 20, 1], [286, 33, 1], [286, 39, 1], [286, 85, 1], [286, 130, 1], [286, 140, 1], [287, 20, 1], [287, 42, 1], [287, 85, 1], [287, 113, 1], [287, 118, 1], [287, 140, 1], [287, 182, 1], [288, 10, 1], [288, 29, 1], [288, 42, 1], [288, 69, 1], [288, 134, 1], [288, 174, 1], [289, 5, 3], [289, 16, 3], [289, 26, 3], [289, 68, 3], [289, 180, 3], [290, 0, 2], [290, 72, 2], [291, 17, 2], [291, 24, 2], [291, 36, 2], [291, 43, 2], [291, 79, 2], [291, 91, 2], [291, 132, 2], [291, 142, 2], [292, 32, 2], [292, 62, 2], [292, 84, 2], [292, 90, 2], [292, 112, 2], [292, 137, 2], [292, 178, 2], [293, 24, 2], [293, 35, 2], [293, 62, 2], [293, 79, 2], [293, 131, 2], [293, 142, 2], [293, 176, 2], [294, 17, 2], [294, 45, 2], [294, 62, 2], [294, 84, 2], [294, 114, 2], [294, 132, 2], [294, 144, 2], [294, 153, 2], [295, 6, 2], [295, 86, 2], [295, 91, 2], [295, 114, 2], [295, 137, 2], [295, 148, 2], [295, 186, 2], [296, 63, 1], [296, 65, 1], [296, 149, 1], [296, 174, 1], [296, 189, 1], [297, 21, 2], [297, 35, 2], [297, 38, 2], [297, 86, 2], [297, 120, 2], [297, 129, 2], [297, 137, 2], [298, 17, 2], [298, 23, 2], [298, 47, 2], [298, 79, 2], [298, 91, 2], [298, 114, 2], [298, 132, 2], [298, 143, 2], [299, 40, 2], [299, 86, 2], [299, 114, 2], [299, 129, 2], [299, 144, 2], [299, 153, 2], [299, 185, 2], [300, 33, 1], [300, 56, 1], [300, 82, 1], [300, 138, 1], [300, 192, 1], [301, 7, 1], [301, 14, 1], [301, 25, 1], [301, 74, 1], [301, 134, 1], [301, 145, 1], [302, 10, 1], [302, 18, 1], [302, 27, 1], [302, 65, 1], [302, 134, 1], [303, 22, 2], [303, 35, 2], [303, 45, 2], [303, 86, 2], [303, 132, 2], [303, 144, 2], [303, 153, 2], [304, 5, 3], [304, 19, 3], [304, 34, 3], [304, 83, 3], [304, 128, 3], [304, 141, 3], [305, 33, 1], [305, 56, 1], [305, 82, 1], [305, 138, 1], [305, 154, 1], [305, 179, 1], [306, 23, 2], [306, 79, 2], [306, 114, 2], [306, 129, 2], [306, 142, 2], [306, 153, 2], [306, 178, 2], [307, 19, 3], [307, 80, 3], [307, 88, 3], [307, 115, 3], [307, 127, 3], [307, 139, 3], [308, 19, 3], [308, 80, 3], [308, 89, 3], [308, 115, 3], [308, 128, 3], [308, 139, 3], [309, 5, 3], [309, 26, 3], [309, 71, 3], [309, 76, 3], [309, 147, 3], [310, 0, 2], [310, 62, 2], [310, 72, 2], [310, 146, 2], [311, 5, 3], [311, 13, 3], [311, 26, 3], [311, 71, 3], [311, 133, 3], [311, 157, 3], [312, 25, 1], [312, 74, 1], [312, 126, 1], [312, 134, 1], [312, 145, 1], [313, 12, 1], [313, 25, 1], [313, 69, 1], [313, 134, 1], [313, 145, 1], [314, 50, 1], [314, 82, 1], [314, 113, 1], [314, 118, 1], [314, 122, 1], [314, 138, 1], [315, 14, 1], [315, 25, 1], [315, 42, 1], [315, 74, 1], [315, 134, 1], [316, 21, 2], [316, 35, 2], [316, 79, 2], [316, 112, 2], [316, 120, 2], [316, 143, 2], [316, 178, 2], [317, 23, 2], [317, 35, 2], [317, 45, 2], [317, 53, 2], [317, 86, 2], [317, 120, 2], [317, 132, 2], [317, 144, 2], [318, 21, 2], [318, 32, 2], [318, 86, 2], [318, 90, 2], [318, 129, 2], [318, 137, 2], [319, 22, 2], [319, 32, 2], [319, 84, 2], [319, 129, 2], [319, 144, 2], [319, 185, 2], [320, 32, 2], [320, 46, 2], [320, 84, 2], [320, 90, 2], [320, 129, 2], [320, 137, 2], [321, 21, 2], [321, 32, 2], [321, 50, 2], [321, 81, 2], [321, 90, 2], [321, 131, 2], [321, 142, 2], [322, 6, 2], [322, 17, 2], [322, 24, 2], [322, 84, 2], [322, 114, 2], [322, 137, 2], [322, 153, 2], [323, 21, 2], [323, 32, 2], [323, 39, 2], [323, 50, 2], [323, 86, 2], [323, 90, 2], [323, 131, 2], [323, 137, 2], [324, 83, 3], [324, 87, 3], [324, 115, 3], [324, 141, 3], [325, 17, 2], [325, 23, 2], [325, 36, 2], [325, 81, 2], [325, 142, 2], [325, 182, 2], [326, 21, 2], [326, 32, 2], [326, 50, 2], [326, 84, 2], [326, 118, 2], [326, 129, 2], [326, 137, 2], [327, 19, 3], [327, 34, 3], [327, 80, 3], [327, 89, 3], [327, 128, 3], [327, 139, 3], [327, 190, 3]]


In [18]:
# State the input parameters for the 'HillClimber' function
XSet = HighMedLowXSet
YSet = HighMedLowYSet
ZSet = HighMedLowZSet
BestScore = -1
FixedX = []
FixedY = []
FixedZ = []
Students = range(n)
Courses = ActualCourses
LearningGroups = [1,2,3]
Triplets = range(len(TripletList))

# Run the HillClimber
for i in range(1):
    start_time = time.time()
    Iteration = HillClimber(BestScore, Students, Courses, Triplets, TripletBlocks, FixedX, FixedY, FixedZ, XSet, YSet, ZSet, 20)
    if Iteration[0]>=BestScore:
        BestScore = Iteration[0]
        XSet = Iteration[1]
        YSet = Iteration[2]
        ZSet = Iteration[3]
        file = open('Final_XSet.txt', 'w')
        file.write(str(XSet))
        file.close()
        file = open('Final_YSet.txt', 'w')
        file.write(str(YSet))
        file.close()
        file = open('Final_ZSet.txt', 'w')
        file.write(str(ZSet))
        file.close()
        solving_time = time.time() - start_time
        print("Step", i+1, "has this many points:", BestScore, "in total time:", round(solving_time,1))


Step 1 has this many points: 11626 in total time: 86.5


In [19]:
# print the optimized schedule

mapping = dict({'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'H':8})
TripletListOrdinal = []
for t in TripletList:
    obj = [t[0], t[1], mapping[t[2]]]
    TripletListOrdinal.append(obj)
TripletListOrdinal


schedule=[ [0,0,0,0,0,0,0,0,0,0,0] for i in Students]
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            if [i,t,l] in ZSet: 
                schedule[i][0]=StudentIDList[i]
                if i in Grade11Students:
                    grade = 11
                if i in Grade12Students:
                    grade = 12
                schedule[i][1]=grade
                schedule[i][2]=l
                schedule[i][TripletListOrdinal[t][2]+2]=TripletList[t][0]
                if TripletListOrdinal[t][0] not in StudentChoices[i]:
                    print("ERROR: student",i, 'did not request course',CourseIDList[TripletListOrdinal[t][0]],
                          '(',CourseNameList[TripletListOrdinal[t][0]], '), in block',TripletListOrdinal[t][2])

schedule[:5]

[[6689, 12, 3, 63, 0, 24, 0, 0, 45, 49, 68],
 [6690, 12, 3, 63, 0, 24, 0, 0, 62, 49, 68],
 [6695, 12, 3, 0, 64, 0, 56, 23, 14, 80, 74],
 [6697, 12, 1, 14, 17, 5, 0, 59, 0, 64, 56],
 [6701, 12, 2, 0, 26, 22, 0, 0, 0, 56, 0]]

### Statistics

In [20]:
# Descriptive Statistics of our results

MustCount=0
HighCount=0
MedCount=0
LowCount=0
MustTotal=0
HighTotal=0
MedTotal=0
LowTotal=0
Courses = ActualCourses


for i in Students:
    for j in Courses:
        if P[i,j]>0 and not j in [15,16]:
            if j in MustPriority: MustTotal +=1
            elif j in HighPriority: HighTotal +=1
            elif j in MedPriority: MedTotal +=1
            else: LowTotal +=1
        
for i in Students:
    for t in Triplets:
        for l in LearningGroups:
            if [i,t,l] in ZSet:
                j = TripletList[t][0]
                if not j in [15,16]:
                    if j in MustPriority: MustCount +=1
                    elif j in HighPriority: HighCount +=1
                    elif j in MedPriority: MedCount +=1
                    else: LowCount +=1
                    
Total = MustTotal+HighTotal+MedTotal+LowTotal
Count = MustCount+HighCount+MedCount+LowCount

print(MustCount, "out of", MustTotal, "mandatory priority course assignments were made:", 
     round(100*MustCount/MustTotal, 2), "percent")
print(HighCount, "out of", HighTotal, "high priority course assignments were made:", 
     round(100*HighCount/HighTotal, 2), "percent")
print(MedCount, "out of", MedTotal, "medium priority course assignments were made:", 
     round(100*MedCount/MedTotal, 2), "percent")
print(LowCount, "out of", LowTotal, "low priority course assignments were made:", 
     round(100*LowCount/LowTotal, 2), "percent")
print("Overall percentage of desired courses assigned to students:", round(100*Count/Total, 2), "percent")

print()
print()
for l in [1,2,3]:
    count11 = 0
    count12 = 0
    for y in YSet:
        i = y[0]
        if y[1]==l: 
            if i in Grade11Students: count11 += 1
            if i in Grade12Students: count12 += 1
    print("Learning Group", l, "has", count11+count12, 'total students (', count11, "grade 11 and",
          count12, "grade 12 )")

807 out of 807 mandatory priority course assignments were made: 100.0 percent
518 out of 541 high priority course assignments were made: 95.75 percent
147 out of 166 medium priority course assignments were made: 88.55 percent
525 out of 789 low priority course assignments were made: 66.54 percent
Overall percentage of desired courses assigned to students: 86.71 percent


Learning Group 1 has 110 total students ( 38 grade 11 and 72 grade 12 )
Learning Group 2 has 108 total students ( 91 grade 11 and 17 grade 12 )
Learning Group 3 has 110 total students ( 36 grade 11 and 74 grade 12 )


In [21]:
print('Objective Function Score:', HighCount*5+MedCount*3+LowCount*1)
print('Solution performance versus our theoretical maximum:', round(Count/(807+518+155+682),2))
print('Solution performance versus total course requests:', round(Count/(807+541+166+789),2))

Objective Function Score: 3556
Solution performance versus our theoretical maximum: 0.92
Solution performance versus total course requests: 0.87


## Sanity Checks
Ignore the code below if you are uninterested in testing the quality of our output timetable

In [22]:
# Check wheter the final timetable assigned any duplicate courses to students
# This should be blank.

for row in schedule:
    for x in range(3,11):
        if row[x]==0:
            pass
        if row[x]!=0:
            if row[3:].count(row[x])>1:
                print('Duplicate found in row', row)

In [23]:
# Double check this is correct
# This should be blank.

for z in ZSet:
    i = z[0]
    t = z[1]
    l = z[2]
    if not [t,l] in XSet:
        print(t,l)
    if not [i,l] in YSet:
        print(i,l)

In [24]:
# This should be blank

for t in Triplets:
    if not t in CombinedLG:
        count=0
        for x in XSet:
            if x[0]==t:
                count+=1
        if count < 1:
            if TripletCap[t]>0:
                if Enrollment[TripletList[t][0]] > 0:
                    print(t, count, TripletCap[t], Enrollment[TripletList[t][0]] )

In [25]:
# These two should be blank

LearningGroups = [1,2,3]

print("These Grade 11 students did not get into Social Studies (they didn't register for the course)")
for i in Grade11Students:
    flag=0
    for t in TripletIDs[60]:
        for l in LearningGroups:
            if [i,t,l] in ZSet:
                flag=1
    if flag==0: print(i, "Student", StudentIDList[i])
        
        
print("")

print("These Grade 11 students did not get their Math Choice")
for i in Grade11Students:
    for j in [12,13,91]:
        if P[i,j]>0:
            flag=0
            for t in TripletIDs[j]:
                for l in LearningGroups:
                    if [i,t,l] in ZSet:
                        flag=1
            if flag==0: print(i, "Student", StudentIDList[i], "did not get into", CourseNameList[j])
        

These Grade 11 students did not get into Social Studies (they didn't register for the course)

These Grade 11 students did not get their Math Choice


In [26]:
# This should just be the numbers from 158 to 173, which are the Grade 11 and 12 spares.

for t in Triplets:
    if t in CombinedLG:
        count=0
        for x in XSet:
            if x[0]==t:
                count+=1
        if count < 3:
            if TripletCap[t]>0:
                if Enrollment[TripletList[t][0]] > 0:
                    print(t, count, TripletCap[t], Enrollment[TripletList[t][0]] )
        

158 0 50 55
159 0 50 55
160 0 50 55
161 0 50 55
162 0 50 55
163 0 50 55
164 0 50 55
165 0 50 55
166 0 50 157
167 0 50 157
168 0 50 157
169 0 50 157
170 0 50 157
171 0 50 157
172 0 50 157
173 0 50 157


In [27]:
print("These students did not get into a MUST course")
for i in Students:
    for j in MustPriority:
        if P[i,j]>1 and P[i,j]!=10:
            flag=0
            for t in TripletIDs[j]:
                for l in LearningGroups:
                    if [i,t,l] in ZSet:
                        flag=1
            if flag==0: print(i, "Student", StudentIDList[i], "did not get into course", CourseNameList[j])
       



These students did not get into a MUST course


In [28]:
for i in [34,231,243,275,286,292,317,319]:
        
    print("")
    print(i, "Here are the courses for Student", StudentIDList[i])
    for l in LearningGroups:
        for t in Triplets:
            if [i,t,l] in ZSet:
                print(StudentIDList[i], "Learning Group", l, "Block", TripletList[t][2], CourseNameList[TripletList[t][0]])
    for j in [54,55,89,90]:
        if j in StudentChoices[i]:
            print("English choice was", CourseNameList[j])
                
                
                                              


34 Here are the courses for Student 6934
6934 Learning Group 3 Block B Economics 12 AP (Micro) (S1)
6934 Learning Group 3 Block D Explorations in Social Studies 11
6934 Learning Group 3 Block F Gov't / Politics 12 AP (Comparative)
6934 Learning Group 3 Block H Literary Studies 11
6934 Learning Group 3 Block A Mandarin 12
6934 Learning Group 3 Block E Physics 11
6934 Learning Group 3 Block G Pre-Calculus 11 - Gr 11
6934 Learning Group 3 Block C Woodwork 11
English choice was Literary Studies 11

231 Here are the courses for Student 8411
8411 Learning Group 3 Block C Anatomy and Physiology 12
8411 Learning Group 3 Block G Explorations in Social Studies 11
8411 Learning Group 3 Block F Latin 11
8411 Learning Group 3 Block H Literary Studies 11
8411 Learning Group 3 Block A Mandarin 12
8411 Learning Group 3 Block D Pre-Calculus 12 - Gr 11
English choice was Literary Studies 11

243 Here are the courses for Student 8461
8461 Learning Group 1 Block C Chemistry 11
8461 Learning Group 1 Block

In [29]:
print("These Grade 11 students did not get into Social Studies (they didn't register for the course)")
for i in Grade11Students:
    flag=0
    for t in TripletIDs[60]:
        for l in LearningGroups:
            if [i,t,l] in ZSet:
                flag=1
    if flag==0: print(i, "Student", StudentIDList[i])
        
        
print("")

print("These Grade 11 students did not get their Math Choice")
for i in Grade11Students:
    for j in [12,13,91]:
        if P[i,j]>0:
            flag=0
            for t in TripletIDs[j]:
                for l in LearningGroups:
                    if [i,t,l] in ZSet:
                        flag=1
            if flag==0: print(i, "Student", StudentIDList[i], "did not get into", CourseNameList[j])

These Grade 11 students did not get into Social Studies (they didn't register for the course)

These Grade 11 students did not get their Math Choice


In [30]:
print("Grade 11 students - statistics for Language Option")
print("")

requested = [0 for j in range(m)]
accepted = [0 for j in range(m)]

nolang=0
for i in Grade11Students:
    flag=0
    for j in [27,34,36,29,30,35,92]:
        if P[i,j]>0: flag=1
    if flag==0:
        nolang+=1
print(nolang, "students requested no Language Course")
print("")
for i in Grade11Students:
    for j in [27,34,36,29,30,35,92]:
        if P[i,j]>=1:
            requested[j] += 1
        if P[i,j]>=1:
            flag=0
            for t in TripletIDs[j]:
                for l in LearningGroups:
                    if [i,t,l] in ZSet:
                        flag=1
            if flag==1: accepted[j] +=1
    
for j in [27,34,36,29,30,35,92]:
    print(CourseNameList[j], "was requested by", requested[j], "with", accepted[j], "accepted")

print("")
print("These Grade 11 students got into a backup Language course")
for i in Grade11Students:
    for j in [27,34,36,29,30,35,92]:
        if P[i,j]==1:
            flag=0
            for t in TripletIDs[j]:
                for l in LearningGroups:
                    if [i,t,l] in ZSet:
                        flag=1
            if flag==1: print(i, "Student", StudentIDList[i], "got into this backup course", 
                              CourseNameList[TripletList[t][0]])

               
            
    

Grade 11 students - statistics for Language Option

12 students requested no Language Course

French 11 was requested by 51 with 51 accepted
Mandarin 11 was requested by 15 with 15 accepted
Spanish 11 was requested by 42 with 42 accepted
French 12 (Honours) was requested by 12 with 11 accepted
French 12 AP was requested by 5 with 4 accepted
Mandarin 12 was requested by 7 with 4 accepted
Mandarin 12 AP was requested by 21 with 15 accepted

These Grade 11 students got into a backup Language course
29 Student 6907 got into this backup course Mandarin 12 AP
34 Student 6934 got into this backup course Mandarin 12
36 Student 6943 got into this backup course Mandarin 12 AP
44 Student 6962 got into this backup course French 12 (Honours)
60 Student 7236 got into this backup course French 12 (Honours)
61 Student 7244 got into this backup course French 12 (Honours)
62 Student 7247 got into this backup course French 12 (Honours)
65 Student 7549 got into this backup course French 12 AP
71 Student 7

In [31]:
for j in [29,30,35,92]:
    print("")
    print(CourseNameList[j])
    print(len(set(ClassList[j]).intersection(Grade11Students)), "Grade 11 students")
    print(len(set(ClassList[j]).intersection(Grade12Students)), "Grade 12 students")


French 12 (Honours)
12 Grade 11 students
5 Grade 12 students

French 12 AP
5 Grade 11 students
5 Grade 12 students

Mandarin 12
7 Grade 11 students
13 Grade 12 students

Mandarin 12 AP
21 Grade 11 students
3 Grade 12 students


In [32]:
# Use the code below to create my Excel output for Jessie

for block in ['A']:
    print("Here are the courses for Block", block)
    print("")
    for t in Triplets:
        if TripletList[t][2] == block:
            LG=[]
            for l in LearningGroups:
                if [t,l] in XSet:
                    LG.append(l)
            Course = CourseNameList[TripletList[t][0]]
            
            count = 0
            for i in Students:
                for l in LearningGroups:
                    if [i,t,l] in ZSet:
                        count+=1
            print(LG, Course, "has enrollment", count, "and capacity", TripletCap[t])            
           


Here are the courses for Block A

[2] 20th Century World History 12 has enrollment 12 and capacity 20
[3] 20th Century World History 12 has enrollment 18 and capacity 20
[1] Chemistry 12 AP has enrollment 19 and capacity 19
[1] Composition 11 has enrollment 0 and capacity 0
[2] Composition 11 has enrollment 17 and capacity 20
[1] Composition 11 has enrollment 18 and capacity 20
[3] English 12 AP (Language) has enrollment 0 and capacity 0
[3] French 11 has enrollment 23 and capacity 24
[2] French 11 has enrollment 18 and capacity 20
[2] French 12 (Honours) has enrollment 11 and capacity 20
[2] French 12 AP has enrollment 6 and capacity 24
[1] Literary Studies 11 has enrollment 20 and capacity 20
[3] Mandarin 12 has enrollment 16 and capacity 22
[2] Mandarin 12 AP has enrollment 16 and capacity 24
[3] Physics 12 has enrollment 8 and capacity 22
[1] Pre-Calculus 12 - Gr 12 has enrollment 15 and capacity 24
[2] Spanish 11 has enrollment 21 and capacity 21
[] Spare 11 has enrollment 0 and c

In [33]:
OurColumns = ["Subject", "Code", "ClassA", "ClassB", "Section", "Block", "Capacity", "High Priority", "Medium Priority",
             "Combined Sections", "Exceptions to Learning Groups", "Running",
              "LG1", "LG2", "LG3", "LG4", "Enrollment", "Total Requested"]

X = [ [ 0 for c in range(18)] for t in Triplets]

for t in Triplets:
    for c in range(12):
        X[t][c] = CourseInfo[t][c]
    j = CourseIDList.index(X[t][1])
    X[t][17] = Enrollment[j]
    
for t in Triplets:
    for l in LearningGroups:
        if [t,l] in XSet: 
            X[t][11+l] = 'Y'
        else: X[t][11+l] = ''
            
for z in ZSet:
    t = z[1]
    X[t][16] += 1

        
        
Output = pd.DataFrame(X, columns=OurColumns)

with pd.ExcelWriter('Optimal SGS Timetable (Courses).xlsx') as writer:  
    Output.to_excel(writer, sheet_name='Courses', index=False)
    
    

In [34]:
# Generate an .xlsx file with the final and near-to-optimal timetable
OurColumns = ["StudentID", "Class", "Day/Board", "Course Name", "Course Code", "Learning Group", "Section", "Block"]

X = [ [ '-' for c in range(8)] for q in range(len(StudentInfo))]

for q in range(len(StudentInfo)):
    for c in range(5):
        X[q][c] = StudentInfo[q][c]

for q in range(len(StudentInfo)):
    myset = []
    i = StudentIDList.index(X[q][0])
    if X[q][4] != 211:
        tlist = TripletIDs[CourseIDList.index(X[q][4])]
        for t in tlist:
            for l in LearningGroups:
                if [i,t,l] in ZSet:
                    X[q][5] = l
                    X[q][6] = TripletList[t][1]
                    X[q][7] = TripletList[t][2] 
        
for q in range(0, len(StudentInfo)-1):
    i0 = StudentIDList.index(X[q][0])
    i1 = StudentIDList.index(X[q+1][0])
    c0 = X[q][4]
    c1 = X[q+1][4]
    j0 = X[q][7]
    j1 = X[q+1][7]
    if i0==i1 and j0==j1 and c0==c1:
        X[q][6] = ''
        X[q][7] = 'Z'

Output = pd.DataFrame(X, columns=OurColumns)

with pd.ExcelWriter('Optimal SGS Timetable (Students).xlsx') as writer:  
    Output.to_excel(writer, sheet_name='Students', index=False)
    